In [14]:
import pandas as pd
import re

# Function to clean the headings
def clean_heading(heading):
    return heading.strip().lower().replace(" ", "")

# Function to remove the last row (Total row) from the DataFrame
def remove_last_row(df):
    return df.iloc[:-1]

# Function to extract the desired values from the "impact" column
def extract_impact(value):
    return value.split("-")[-1].strip()

# Function to extract the first 4 characters from the "eventname" column
def extract_year(eventname):
    return eventname[:4]

# Function to remove text within parentheses for "region" column
def remove_parentheses(data):
    return re.sub(r'\([^)]*\)', '', data)

# Load data from CSV files
file_path1 = "TEST1.csv"
file_path2 = "TEST2.csv"

data1 = pd.read_csv(file_path1)
data2 = pd.read_csv(file_path2)

# Clean the headings by removing spaces
data1.columns = [clean_heading(col) for col in data1.columns]
data2.columns = [clean_heading(col) for col in data2.columns]

# If "ModelName" heading exists in data1, rename it to "Region"
if "modelname" in data1.columns:
    data1.rename(columns={"modelname": "region"}, inplace=True)

# Remove the last row from each DataFrame
data1 = remove_last_row(data1)
data2 = remove_last_row(data2)

# Extract only the words "Direct," "Indirect," and "Induced" from the "impact" column
data1["impact"] = data1["impact"].apply(extract_impact)
data2["impact"] = data2["impact"].apply(extract_impact)

# Combine the data into one DataFrame
combined_data = pd.concat([data1, data2], ignore_index=True)

# Remove text within parentheses in the "region" column
combined_data["region"] = combined_data["region"].apply(remove_parentheses)

# Extracting the year and creating a new "year" column
combined_data["year"] = combined_data["eventname"].apply(extract_year)

# Deleting unwanted columns
columns_to_delete = ["groupname"]
combined_data.drop(columns=columns_to_delete, inplace=True)

# Re-ordering columns as desired
desired_order = ["region","year", "eventname", "impact", "employment", "laborincome", "valueadded","output"]
combined_data = combined_data[desired_order]

# Save the combined data to a new CSV file
output_file_path = "combined_data.csv"
combined_data.to_csv(output_file_path, index=False)

print("Data cleaning, re-ordering, year extraction, and combining completed. Combined data saved to 'combined_data.csv'.")


Data cleaning, re-ordering, year extraction, and combining completed. Combined data saved to 'combined_data.csv'.


#### The code below has not been tested and is based on the template above but adapted to take inputs from all states with electrical cooperatives.

In [ ]:
import pandas as pd
import re
import os

# Function to clean the headings
def clean_heading(heading):
    return heading.strip().lower().replace(" ", "")

# Function to remove the last row (Total row) from the DataFrame
def remove_last_row(df):
    return df.iloc[:-1]

# Function to extract the desired values from the "impact" column
def extract_impact(value):
    return value.split("-")[-1].strip()

# Function to extract the first 4 characters from the "eventname" column
def extract_year(eventname):
    return eventname[:4]

# Function to remove text within parentheses for "region" column
def remove_parentheses(data):
    return re.sub(r'\([^)]*\)', '', data)

# List of all states in the USA (excluding Massachusetts and Connecticut)
STATES = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Delaware", "Florida", "Georgia",
          "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
          "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
          "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
          "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
          "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

# Create an empty list to store the combined data from all files
combined_data_list = []

# Loop through all files in the directory with the specified naming pattern
for filename in os.listdir("."):
    if filename.endswith(".csv") and re.match(r'^\d{8}-(.*?)-Summary Economic Indicators$', filename):
        state_name = re.search(r'^\d{8}-(.*?)-Summary Economic Indicators$', filename).group(1)
        
        if state_name in STATES:
            # Load data from the current CSV file
            data = pd.read_csv(filename)

            # Clean the headings by removing spaces and text within parentheses for "region"
            data.columns = [clean_heading(col) for col in data.columns]

            # Remove the last row from the DataFrame
            data = remove_last_row(data)

            # Extract only the words "Direct," "Indirect," and "Induced" from the "impact" column
            data["impact"] = data["impact"].apply(extract_impact)

            # Remove text within parentheses in the "region" column
            data["region"] = data["region"].apply(remove_parentheses)

            # Extracting the year and creating a new "year" column
            data["year"] = data["eventname"].apply(extract_year)

            # Deleting unwanted column "groupname"
            data.drop(columns=["groupname"], inplace=True)

            # Reordering columns as desired
            desired_order = ["region", "year", "eventname", "impact", "employment", "laborincome", "valueadded", "output"]
            data = data[desired_order]

            # Append the cleaned data to the combined_data_list
            combined_data_list.append(data)

# Combine all DataFrames from the list into one DataFrame
combined_data = pd.concat(combined_data_list, ignore_index=True)

# Save the combined data to a new CSV file
output_file_path = "combined_data.csv"
combined_data.to_csv(output_file_path, index=False)

print("Data cleaning, re-ordering, year extraction, and combining completed. Combined data saved to 'combined_data.csv'.")
